In [79]:
import json
import re
import random

In [2]:
f = open('../Data/fbdata.json', "r")

In [3]:
fb_corpus = json.load(f)

In [4]:
f = open('../Data/people.popular.2', 'rb')
pop_data = f.readlines()

In [105]:
# create a corpus of the top N person in the freebase
random.seed(1)
top_n = 100
top_list = [unicode(re.split(r'\t+', pop)[0], 'utf8') for pop in pop_data[0:top_n*1000]]
selected_list = random.sample(top_list, top_n*2)
used_names = set()
top_fb_corpus = {}

for key, val in fb_corpus.iteritems():
    if val.get(u'name') in selected_list and not val.get(u'name') in used_names:
        top_fb_corpus[key] = val
        used_names.add(val.get(u'name'))
    if len(top_fb_corpus) >= top_n:
        break
print len(top_fb_corpus)

100


In [106]:
len(pop_data)

155846

Analysis the top N people a little bit

In [107]:
# get all the fileds
field_set = set()
field_count = {}
field_modality = {}
empty_cnt = {}
for person_key, person_attributes in top_fb_corpus.iteritems():
    for field, content in person_attributes.iteritems():
        # update count and update modality
        moda = field_modality.get(field, set())
        cnt = field_count.get(field, 0)
        emptyCnt = empty_cnt.get(field,0)
        if type(content) is list:
            cnt += len(content)
            for c in content:
                moda.add(c)
            if len(content) == 0:
                emptyCnt += 1
        else:
            cnt += 1
            moda.add(content)  

        # save
        field_set.add(field)  
        field_count[field] = cnt
        field_modality[field] = moda
        empty_cnt[field] = emptyCnt
print str(len(field_set)) + ' unique fileds'
print ''
print [(k, v/float(top_n)) for k, v in field_count.iteritems()]
print ''
print [(k, len(v)) for k, v in field_modality.iteritems()]
print ''
print [(k, v/float(top_n)) for k, v in empty_cnt.iteritems()]

11 unique fileds

[(u'birthplace', 0.67), (u'education', 0.48), (u'name', 1.0), (u'degree', 0.03), (u'gender', 0.81), (u'profession', 1.52), (u'birthday', 0.67), (u'deathplace', 0.14), (u'nationality', 0.62), (u'spouse', 0.6), (u'Types', 4.97)]

[(u'birthplace', 64), (u'education', 45), (u'name', 100), (u'degree', 3), (u'gender', 2), (u'profession', 61), (u'birthday', 67), (u'deathplace', 14), (u'nationality', 18), (u'spouse', 52), (u'Types', 83)]

[(u'birthplace', 0.33), (u'education', 0.7), (u'name', 0.0), (u'degree', 0.98), (u'gender', 0.19), (u'profession', 0.28), (u'birthday', 0.33), (u'deathplace', 0.86), (u'nationality', 0.38), (u'spouse', 0.78), (u'Types', 0.0)]


save the top_corpus

In [110]:
json.dump(top_fb_corpus, open('../Data/top100.json', 'w'))

In [109]:
empty_cnt

{u'Types': 0,
 u'birthday': 33,
 u'birthplace': 33,
 u'deathplace': 86,
 u'degree': 98,
 u'education': 70,
 u'gender': 19,
 u'name': 0,
 u'nationality': 38,
 u'profession': 28,
 u'spouse': 78}